✅ v3 코드 반영 완료.  
다음 항목이 포함됨:

- 법령명, 개정유형, 의안번호, 발의일 → extract_meta()로 한 번에 추출

- 전부개정 예외처리 포함

- 신구조문 비교 결과는 CSV에 저장되지만, 메타데이터는 따로 dict 형태로 반환

- 함수 process_single_pdf()가 전체 흐름 처리

In [2]:
import re
import pdfplumber
import csv
from difflib import SequenceMatcher
import os


def extract_text_from_pdf(path):
    with pdfplumber.open(path) as pdf:
        text = "\n".join(
            page.extract_text() for page in pdf.pages if page.extract_text()
        )
    return text


def extract_meta(text):
    meta = {}
    law_match = re.search(r"([\w\d가-힣]+법)\s*(일부|전부)?개정법률안", text)
    if law_match:
        meta["법령명"] = law_match.group(1)
        meta["개정유형"] = law_match.group(2)
    bill_match = re.search(r"의안[\s:：]*번호[\s:：]*(제?\d+호)", text)
    if bill_match:
        meta["의안번호"] = bill_match.group(1)
    date_match = re.search(r"발의[\s:：]*연월일[\s:：]*([\d\.\-]+)", text)
    if date_match:
        meta["발의연월일"] = date_match.group(1)
    return meta


def extract_table_section(text):
    lines = text.splitlines()
    start_idx = next((i for i, line in enumerate(lines) if "신" in line and "구" in line), 0)
    return "\n".join(lines[start_idx:])


def split_left_right(text):
    lines = text.splitlines()
    left_lines, right_lines = [], []
    midpoint = max(len(line) for line in lines) // 2
    for line in lines:
        left = line[:midpoint].strip()
        right = line[midpoint:].strip()
        left_lines.append(left)
        right_lines.append(right)
    return "\n".join(left_lines), "\n".join(right_lines)


def split_by_clause(text):
    pattern = r"(제\d+조(?:\s*제\d+항)?(?:\s*제\d+호)?(?:\s*제\d+목)?)"
    parts = re.split(pattern, text)
    clauses = []
    for i in range(1, len(parts), 2):
        law_id = parts[i].strip()
        law_body = parts[i + 1].strip() if i + 1 < len(parts) else ""
        clauses.append((law_id, law_body))
    return clauses


def clean_text(text):
    remove_keywords = ["현행과 같음", "------", "생략"]
    lines = text.splitlines()
    lines = [line.strip() for line in lines if line.strip() and not any(k in line for k in remove_keywords)]
    return " ".join(lines)


def compare_clauses_v2(old_clauses, new_clauses, similarity_threshold=0.85):
    results = []
    old_dict = {cid: clean_text(text) for cid, text in old_clauses}
    new_ids = set(cid for cid, _ in new_clauses)

    for cid, new_text_raw in new_clauses:
        new_text = clean_text(new_text_raw)
        old_text = old_dict.get(cid)
        if old_text:
            sim = SequenceMatcher(None, old_text, new_text).ratio()
            if sim < similarity_threshold:
                results.append((cid, old_text, new_text, "수정"))
        else:
            results.append((cid, "", new_text, "신설"))

    for cid, old_text_raw in old_clauses:
        if cid not in new_ids:
            old_text = clean_text(old_text_raw)
            results.append((cid, old_text, "", "삭제"))

    return results


def save_to_csv(data, path):
    with open(path, mode="w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["조문 ID", "구조문", "신조문", "변경유형"])
        for row in data:
            writer.writerow(list(row))


def process_single_pdf(pdf_path, output_csv):
    text = extract_text_from_pdf(pdf_path)
    meta = extract_meta(text)
    if meta.get("개정유형") == "전부":
        print(f"[예외처리] {meta.get('법령명', '알 수 없음')}은 전부개정으로 비교 생략")
        return meta

    table_section = extract_table_section(text)
    old_text, new_text = split_left_right(table_section)
    old_clauses = split_by_clause(old_text)
    new_clauses = split_by_clause(new_text)
    result = compare_clauses_v2(old_clauses, new_clauses)
    save_to_csv(result, output_csv)
    return meta

In [5]:
# === 예시 ===
meta_info = process_single_pdf("../data/no_upload/2210568_의사국 의안과_의안원문.pdf", "../data/processed/조문_비교결과_v3.csv")


In [6]:
import pandas as pd

file_path = "../data/processed/조문_비교결과_v3.csv"
df = pd.read_csv(file_path)

df.to_excel("../data/processed/조문_비교결과_v3.xlsx", index=False)

In [7]:
df = pd.read_csv(file_path)

df.head()

df.columns

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   조문 ID   3 non-null      object
 1   구조문     3 non-null      object
 2   신조문     1 non-null      object
 3   변경유형    3 non-null      object
dtypes: object(4)
memory usage: 228.0+ bytes


엑셀 구성에 맞춰 코드 업데이트 완료 ✅

변경사항:

각 조문 ID를 조 / 항 / 호 / 목으로 분리

CSV 컬럼: 법령명, 개정유형, 조문ID, 조, 항, 호, 목, 구조문, 신조문, 변경유형

compare_clauses_v3()와 save_to_csv_v3() 반영

전부개정 예외처리 유지

In [8]:
import re
import pdfplumber
import csv
from difflib import SequenceMatcher
import os


def extract_text_from_pdf(path):
    with pdfplumber.open(path) as pdf:
        text = "\n".join(
            page.extract_text() for page in pdf.pages if page.extract_text()
        )
    return text


def extract_meta(text):
    meta = {}
    law_match = re.search(r"([\w\d가-힣]+법)\s*(일부|전부)?개정법률안", text)
    if law_match:
        meta["법령명"] = law_match.group(1)
        meta["개정유형"] = law_match.group(2)
    bill_match = re.search(r"의안[\s:：]*번호[\s:：]*(제?\d+호)", text)
    if bill_match:
        meta["의안번호"] = bill_match.group(1)
    date_match = re.search(r"발의[\s:：]*연월일[\s:：]*([\d\.\-]+)", text)
    if date_match:
        meta["발의연월일"] = date_match.group(1)
    return meta


def extract_table_section(text):
    lines = text.splitlines()
    start_idx = next((i for i, line in enumerate(lines) if "신" in line and "구" in line), 0)
    return "\n".join(lines[start_idx:])


def split_left_right(text):
    lines = text.splitlines()
    left_lines, right_lines = [], []
    midpoint = max(len(line) for line in lines) // 2
    for line in lines:
        left = line[:midpoint].strip()
        right = line[midpoint:].strip()
        left_lines.append(left)
        right_lines.append(right)
    return "\n".join(left_lines), "\n".join(right_lines)


def split_by_clause(text):
    pattern = r"(제\d+조(?:\s*제\d+항)?(?:\s*제\d+호)?(?:\s*제\d+목)?)"
    parts = re.split(pattern, text)
    clauses = []
    for i in range(1, len(parts), 2):
        law_id = parts[i].strip()
        law_body = parts[i + 1].strip() if i + 1 < len(parts) else ""
        clauses.append((law_id, law_body))
    return clauses


def clean_text(text):
    remove_keywords = ["현행과 같음", "------", "생략"]
    lines = text.splitlines()
    lines = [line.strip() for line in lines if line.strip() and not any(k in line for k in remove_keywords)]
    return " ".join(lines)


def parse_clause_id(clause_id):
    jo = hang = ho = mok = ""
    m = re.match(r"제(\d+)조(?:\s*제(\d+)항)?(?:\s*제(\d+)호)?(?:\s*제(\d+)목)?", clause_id)
    if m:
        jo, hang, ho, mok = m.groups()
    return jo or "", hang or "", ho or "", mok or ""


def compare_clauses_v3(old_clauses, new_clauses, law_name, revision_type, threshold=0.85):
    results = []
    old_dict = {cid: clean_text(text) for cid, text in old_clauses}
    new_ids = set(cid for cid, _ in new_clauses)

    for cid, new_text_raw in new_clauses:
        new_text = clean_text(new_text_raw)
        old_text = old_dict.get(cid)
        jo, hang, ho, mok = parse_clause_id(cid)
        if old_text:
            sim = SequenceMatcher(None, old_text, new_text).ratio()
            if sim < threshold:
                results.append([law_name, revision_type, cid, jo, hang, ho, mok, old_text, new_text, "수정"])
        else:
            results.append([law_name, revision_type, cid, jo, hang, ho, mok, "", new_text, "신설"])

    for cid, old_text_raw in old_clauses:
        if cid not in new_ids:
            old_text = clean_text(old_text_raw)
            jo, hang, ho, mok = parse_clause_id(cid)
            results.append([law_name, revision_type, cid, jo, hang, ho, mok, old_text, "", "삭제"])

    return results


def save_to_csv_v3(data, path):
    with open(path, mode="w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["법령명", "개정유형", "조문ID", "조", "항", "호", "목", "구조문", "신조문", "변경유형"])
        for row in data:
            writer.writerow(row)


def process_single_pdf_v3(pdf_path, output_csv):
    text = extract_text_from_pdf(pdf_path)
    meta = extract_meta(text)
    if meta.get("개정유형") == "전부":
        print(f"[예외처리] {meta.get('법령명', '알 수 없음')}은 전부개정으로 비교 생략")
        return meta

    law_name = meta.get("법령명", "")
    revision_type = meta.get("개정유형", "")

    table_section = extract_table_section(text)
    old_text, new_text = split_left_right(table_section)
    old_clauses = split_by_clause(old_text)
    new_clauses = split_by_clause(new_text)
    result = compare_clauses_v3(old_clauses, new_clauses, law_name, revision_type)
    save_to_csv_v3(result, output_csv)
    return meta

In [9]:
meta_info = process_single_pdf_v3("../data/no_upload/2210568_의사국 의안과_의안원문.pdf", "../data/processed/조문_비교결과_v3_2.csv")


In [10]:
df = pd.read_csv(file_path)

df.head()

df.columns

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   조문 ID   3 non-null      object
 1   구조문     3 non-null      object
 2   신조문     1 non-null      object
 3   변경유형    3 non-null      object
dtypes: object(4)
memory usage: 228.0+ bytes


In [11]:
file_path = "../data/processed/조문_비교결과_v3_2.csv"
df = pd.read_csv(file_path)

df.to_excel("../data/processed/조문_비교결과_v3_2.xlsx", index=False)

# ❌ v3 실패 원인 분석: "상가건물 임대차보호법" 사례

## 📝 요약
- 문서: `2210568_의사국 의안과_의안원문.pdf`
- 문제: **실제 변경 사항이 추출되지 않거나, 변경 유형 오인**

---

## 📉 실패 원인 목록

### 1. **신·구조문대비표 구간 추출 미흡**
- 문제: `extract_table_section()`이 `"신"`과 `"구"` 포함 라인을 기준으로 잘라서 **부정확한 시작점에서 시작**
- 결과: 신·구 조문이 아닌 본문 일부가 포함되거나 잘림

---

### 2. **좌/우 분리 실패**
- 문제: `split_left_right()`가 **고정 `midpoint` 기준 자르기**
- 실제 PDF는 줄마다 좌우 폭이 달라, **구조문/신조문이 섞임**
- 특히 `제19조의2` 신설 조문은 오른쪽 전체에만 있음 → **좌측에도 남는 텍스트 발생**

---

### 3. **조문 구분 정확도 저하**
- 문제: `split_by_clause()`의 정규식이 **"제X조"만 인식**하고 이후 항, 호, 목은 제대로 대응 안 됨
- 조문 없이 이어지는 문단은 **묶이지 않거나 잘림**

---

### 4. **유사도 기반 비교의 한계**
- 문제: `SequenceMatcher`는 간단한 문장 비교에 적합하지만,
  - 조문 텍스트 길이가 짧고 핵심 변경만 있어도 **수정인지 신설인지 구분 실패**
- 예: `분담` → `분담, 관리비 부과 항목` → `수정` 인식 못 함

---

### 5. **신설 조문 누락**
- 문제: 신설된 `제19조의2`는 **좌측에 조문 ID가 없음**
- `split_by_clause()`가 이를 인식 못해 **조문 ID 없이 누락되거나 쓰레기값으로 처리**

---

## 📌 개선 포인트 (v4 계획 요소)

| 실패 원인 | 개선 방안 |
|-----------|-----------|
| 신·구 구조문 구간 탐색 | "신·구조문대비표" 키워드와 위치 기반 탐지 |
| 좌우 분리 | `pdfplumber`의 `char.bbox` 또는 `layout` 정보 활용 |
| 조문 추출 | "제X조 제X항…" 패턴 강화 + 조문 없음 감지 로직 추가 |
| 비교 로직 | `SequenceMatcher` → `BLEU` or `Jaccard`, 또는 LLM 요약 기반 비교 |
| 신설 누락 방지 | 조문 ID 없음 → '신설 후보'로 태깅하여 수동검토 목록 포함 |

---

## 📂 결론
- 현재 파이프라인은 "한쪽이 완전히 비어있거나 구조가 다를 경우" 완전 실패
- `좌우 분리 방식`, `정규식 기반 조문 추출`, `간단한 유사도 비교` → **다음 버전에서 반드시 보완 필요**

